**GSPLAT**

In [ ]:
!nvcc --version

In [ ]:
!git clone https://github.com/nerfstudio-project/gsplat.git

In [ ]:
!rm -rf "/content/gsplat/gsplat/cuda/csrc/third_party/glm/"

In [ ]:
!git clone https://github.com/g-truc/glm.git

!mv glm "/content/gsplat/gsplat/cuda/csrc/third_party"

In [ ]:
import sys, os

sys.path.insert(0, "/content/gsplat")
sys.path.insert(0, "/content/gsplat/gsplat")
sys.path.insert(0, "/content/gsplat/examples")
sys.path.append(0, "/content/gsplat/examples/datasets")
sys.path.insert(0, "/content/gsplat/gsplat/cuda/csrc/third_party/glm")

In [ ]:
import sys
sys.path

In [ ]:
%cd /content/gsplat

In [ ]:
"""import sys, os
#sys.path.insert(0, "/content/vcpkg")
sys.path.insert(0, "/content/gsplat/examples")
sys.path.insert(0, "/content/gsplat/examples/datasets")
sys.path.insert(0, "/content/gsplat/gsplat/rendering.py")
"""

In [ ]:
!pip install gsplat

In [ ]:
%cd /content/gsplat

**Set Configuration**

In [ ]:
!pip install ninja numpy jaxtyping rich
!pip install gsplat --index-url https://docs.gsplat.studio/whl/pt20cu118

In [ ]:
!pip install -r examples/requirements.txt

In [ ]:
import os
print(os.getcwd())

**Set Configuration**

In [ ]:
"""
from simple_trainer import Config

# Create config object with overrides
cfg = Config(
    data_dir="/content/my_colmap_dataset",
    result_dir="/content/results/my_scene",
    data_factor=1,
    max_steps=30_000,
    save_steps=[7_000, 30_000],
    save_ply=True,
    ply_steps=[7_000, 30_000],
    init_num_pts=100_000,
    pose_opt=True,
    app_opt=True,
    disable_viewer=True
)
print(cfg)
"""

**COLMAP**

In [ ]:
# @title Frames Extraction Pipeline
import cv2
import os

def extract_frames(video_path, output_dir, frames_per_second=5):

    # Ensure output folder exists
    os.makedirs(output_dir, exist_ok=True)

    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video: {video_path}")

    # Video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_step = int(max(1, fps // frames_per_second))  # Step between frames

    frame_count, saved_count = 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame at interval
        if frame_count % frame_step == 0:
            filename = os.path.join(output_dir, f"frame_{saved_count:05d}.jpg")
            cv2.imwrite(filename, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f" Saved {saved_count} frames in '{output_dir}'")


video_path = "path/to/input_video.mp4"
output_dir = "/content/gsplat/data/Room_Data/images"
frames_per_second = 5  # Frames to extract per second

extract_frames(video_path, output_dir, frames_per_second)

In [ ]:
!sudo apt-get install colmap

In [ ]:
!colmap -h

In [ ]:
import os
os.environ["QT_QPA_PLATFORM"] = "offscreen"

# Paths
IMAGE_PATH="/content/gsplat/data/Room_Data/images"
SPARSE_PATH="/content/gsplat/data/Room_Data/sparse"
DATABASE_PATH="/content/gsplat/data/Room_Data/database.db"

# Create directories
!mkdir -p $SPARSE_PATH
!mkdir -p $(dirname $DATABASE_PATH)

# Feature extraction
!colmap feature_extractor \
    --database_path $DATABASE_PATH \
    --image_path $IMAGE_PATH \
    --SiftExtraction.use_gpu 0 \
    --SiftExtraction.estimate_affine_shape 1 \
    --SiftExtraction.domain_size_pooling 1

# Feature matching
!colmap exhaustive_matcher \
    --database_path $DATABASE_PATH \
    --SiftMatching.use_gpu 0

# Sparse reconstruction (SfM mapper)
!colmap mapper \
    --database_path $DATABASE_PATH \
    --image_path $IMAGE_PATH \
    --output_path $SPARSE_PATH

**TRAINING GSPLAT**

In [ ]:
!PYTHONPATH=$(pwd):$PYTHONPATH CUDA_VISIBLE_DEVICES=0 \     #CUDA_VISIBLE_DEVICES=0 - Set Accordingly
  python examples/simple_trainer.py default \
  --data_dir /content/gsplat/data/Room_Data \
  --data_factor 1 \
  --result_dir /content/results/my_dataset